In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
source = pd.read_excel("12-31-2023_Rocket_Simulated_Data.xlsx")

In [3]:
# Stops program once below lower invincibility frame
class StopExecution(Exception):
    def _render_traceback(self):
        return []

In [6]:
i_frame_upper = float(input("Enter upper invincibility altitude: "))
i_frame_lower = float(input("Enter lower invincibility altitude: "))
poll_interval = float(input("Enter poll interval in seconds: "))
in_i_frame_upper = False
past_upper = False
prev_poll_time = 0

# Resets output file
with open("output.txt","w") as t:
    t.close()


for intervals in range(1, 99999999999999999):

    # Testing: creates subsets of data to simulate polling intervals
    source_sim = source.dropna()
    source_sim = source_sim[source_sim['# Time (s)'] <= (poll_interval * intervals)]
    # End of testing

    # START OF SCRIPT - open file
    with open("output.txt", "a") as target:
        df = source_sim.dropna()
        # For testing purposes- log poll time
        target.write(f"POLL AT {poll_interval * (intervals-1)} SECONDS\n")


        for index, row in df[df['# Time (s)'] > prev_poll_time].iterrows():
            # Ship enters upper invincibility altitude, stop recording
            if row['Altitude (m)'] >= i_frame_upper and past_upper == False and in_i_frame_upper == False:
                target.write("ENTERED UPPER INVINCIBILITY ALTITUDE-PAUSING RECORDING\n")
                in_i_frame_upper = True
            # Ship exits upper invincibility altitude, resume recording
            elif in_i_frame_upper == True and row['Altitude (m)'] < i_frame_upper:
                target.write("EXITED UPPER INVINCIBILITY ALTITUDE-RESUMING RECORDING\n")
                in_i_frame_upper = False
                past_upper = True
                target.write(f"{row['# Time (s)']}\t{row['Altitude (m)']}\t{row['Position East of launch (m)']}\t{row['Position North of launch (m)']}\t{row['Latitude (°)']}\t{row['Longitude (°)']}\n")
            # If ship enters lower invincibility altitude after having reached upper invincibility altitude, close file and stop recording
            elif past_upper == True and row['Altitude (m)'] <= i_frame_lower:
                target.write("ENTERED LOWER INVINCIBILITY ALTITUDE-END\n")
                target.close()
                raise StopExecution
            elif in_i_frame_upper == False:
                target.write(f"{row['# Time (s)']}\t{row['Altitude (m)']}\t{row['Position East of launch (m)']}\t{row['Position North of launch (m)']}\t{row['Latitude (°)']}\t{row['Longitude (°)']}\n")


        prev_poll_time = df.iloc[-1]['# Time (s)']

        # Simulate delay
        time.sleep(poll_interval)
        print("INT")
        # Close file
        target.close()
        

Enter upper invincibility altitude: 4000
Enter lower invincibility altitude: 1000
Enter poll interval in seconds: 2
INT
INT
INT
INT
INT
INT
INT


KeyboardInterrupt: 